In [1]:
import re
from collections import defaultdict
import sympy

In [2]:
# From day 18 with a couple of additions
class Processor():
    def __init__(self, name, instructions, max_moves=None):
        self.instruction_re = re.compile('^([a-z]{3}) ([-0-9a-z]+)(?: ([-0-9a-z]+))?')
        self.registers = defaultdict(lambda: 0)
        self.instruction_position = 0
        self.rcv_queue = []
        self.wait_register = None
        self.partner=None
        self.muls = 0
        self.moves = 0
        self.name = name
        self.instructions = instructions
        self.max_moves = max_moves

    def set_partner(self, partner):
        self.partner = partner

    def get_value(self, rhs):
        try:
            return int(rhs)
        except ValueError:
            return self.registers[rhs]
        
    def set_register(self, lhs, rhs):
        self.registers[lhs] = rhs

    def st(self, lhs, rhs):
        self.instruction_position += 1
        self.registers[lhs] = self.get_value(rhs)
        
    def add(self, lhs, rhs):
        self.instruction_position += 1
        self.registers[lhs] += self.get_value(rhs)

    def sub(self, lhs, rhs):
        self.instruction_position += 1
        self.registers[lhs] -= self.get_value(rhs)

    def mod(self, lhs, rhs):
        self.instruction_position += 1
        self.registers[lhs] %= self.get_value(rhs)

    def mul(self, lhs, rhs):
        self.instruction_position += 1
        self.registers[lhs] *= self.get_value(rhs)
        self.muls += 1
       
    def jgz(self, lhs, rhs):
        if self.get_value(lhs) > 0:
            self.instruction_position += self.get_value(rhs)
        else:
            self.instruction_position += 1

    def jnz(self, lhs, rhs):
        if self.get_value(lhs) != 0:
            self.instruction_position += self.get_value(rhs)
        else:
            self.instruction_position += 1
            
    def snd(self, rhs):
        self.snds += 1
        # print('{} sent {}'.format(self.name, self.sends))
        self.instruction_position += 1
        self.partner.message(self.get_value(rhs))
        
    def rcv(self, lhs=None):
        if not lhs and not self.wait_register:
            raise ValueError('Message sent to terminated SoundRegister {}'.format(self.name))
        
        if lhs:
            self.rcvs += 1
        if self.rcv_queue:
            register = lhs or self.wait_register
            self.registers[register] = self.rcv_queue.pop()
            self.instruction_position += 1
            self.wait_register = None
        else:
            self.wait_register = lhs

    def message(self, value):
        self.rcv_queue.insert(0, value)
        if self.wait_register:
            self.rcv()
            self.play()

    def play(self, log=None):
        instruction_map = {
            'set': (self.st, 2),
            'add': (self.add, 2),
            'sub': (self.sub, 2),
            'mod': (self.mod, 2),
            'mul': (self.mul, 2),
            'snd': (self.snd, 1),
            'rcv': (self.rcv, 1),
            'jgz': (self.jgz, 2),
            'jnz': (self.jnz, 2),
        }

        while self.instruction_position >= 0 and self.instruction_position < len(self.instructions):
            instruction = self.instructions[self.instruction_position]
            m = self.instruction_re.match(instruction)
            if not m:
                raise ValueError('Invalid instruction {}'.format(instruction))
            if m[1] not in instruction_map:
                print('Command: ', m[1])
                raise ValueError('Invalid command {}'.format(instruction))
            cmd, args = instruction_map[m[1]]
            self.moves += 1
            if log and self.moves >= log:
                print('{} {}'.format(self.moves, instruction))
            if args == 2:
                cmd(m[2], m[3])
                if m[2] in ('i'):
                    print('Move {} Register {}: {}'.format(self.moves, m[2], self.registers[m[2]]))
            else:
                cmd(m[2])
            if self.wait_register:
                break
                
            if self.max_moves and self.moves > self.max_moves:
                print('Exceeded max moves', self.name)
                break

In [3]:
with open('coprocessor.txt') as fh:
    instructions = fh.readlines()
instructions = [i.strip() for i in instructions]

In [4]:
# Look at the beginning sequence
processor = Processor('p0', instructions)
processor.play()

In [5]:
processor.muls

9409

In [6]:
# Look at the beginning sequence
processor = Processor('p0', instructions, max_moves=30)
processor.set_register('a', 1)
processor.play(log=1)

1 set b 99
2 set c b
3 jnz a 2
4 mul b 100
5 sub b -100000
6 set c b
7 sub c -17000
8 set f 1
9 set d 2
10 set e 2
11 set g d
12 mul g e
13 sub g b
14 jnz g 2
15 sub e -1
16 set g e
17 sub g b
18 jnz g -8
19 set g d
20 mul g e
21 sub g b
22 jnz g 2
23 sub e -1
24 set g e
25 sub g b
26 jnz g -8
27 set g d
28 mul g e
29 sub g b
30 jnz g 2
31 sub e -1
Exceeded max moves p0


Register `f` probe:

```
Move 8 Register f: 1
Move 439599 Register f: 0
Move 1978178 Register f: 0
Move 2813408 Register f: 0
Move 4521547 Register f: 0
Move 7121435 Register f: 0
Move 10613072 Register f: 0
Move 15869367 Register f: 0
Move 20256521 Register f: 0
Move 22890321 Register f: 0
Move 29038365 Register f: 0
```

In [7]:
f_values = [
    8,
    439599,
    1978178,
    2813408,
    4521547,
    7121435,
    10613072,
    15869367,
    20256521,
    22890321,
    29038365,
]
[print(a-b) for (a, b) in zip(f_values[1:], f_values[:-1])]

439591
1538579
835230
1708139
2599888
3491637
5256295
4387154
2633800
6148044


[None, None, None, None, None, None, None, None, None, None]

In debug mode h is only set once:
First instructions:
```
1 set b 99
2 set c b
3 jnz a 2
4 jnz 1 5

Both carry on from here...
5 set f 1
6 set d 2
7 set e 2
```

Prod mode:
    
```
1 set b 99
2 set c b
3 jnz a 2
4 mul b 100
5 sub b -100000
6 set c b
7 sub c -17000

Both carry on from here...
8 set f 1
9 set d 2
10 set e 2
11 set g d
12 mul g e
```

Initial values:

```
| Cell | Debug | Prod   |
+------+-------+--------+
| a    | 0     | 1      |
| b    | 99    | 109900 |
| c    | 99    | 126900 |
| d    | 2     | 2      |
| e    | 2     | 2      |
| f    | 1     | 1      |
| g    | 4     | 4      |
```

```
B:
sub b -17

D:
set d 2
sub d -1

E:
set e 2
sub e -1

F:
set f 1
set f 0
jnz f 2

G:
set g d
mul g e
sub g b
jnz g 2
set g e
sub g b
jnz g -8
set g d
sub g b
jnz g -13
set g b
sub g c
jnz g 2

H:
sub h -1
```

All about G?



In [8]:
# How does the process terminate - there is a jump back at the end?
processor = Processor('p0', instructions)
processor.play(log=75743)

75743 jnz g 2
75744 sub e -1
75745 set g e
75746 sub g b
75747 jnz g -8
75748 set g d
75749 mul g e
75750 sub g b
75751 jnz g 2
75752 sub e -1
75753 set g e
75754 sub g b
75755 jnz g -8
75756 set g d
75757 mul g e
75758 sub g b
75759 jnz g 2
75760 sub e -1
75761 set g e
75762 sub g b
75763 jnz g -8
75764 sub d -1
75765 set g d
75766 sub g b
75767 jnz g -13
75768 jnz f 2
75769 sub h -1
75770 set g b
75771 sub g c
75772 jnz g 2
75773 jnz 1 3


Last sequence:

```
...
jnz f 2
sub h -1
set g b
sub g c
jnz g 2
jnz 1 3
sub b -17
jnz 1 -23

To terminate g == 0
g == 0 => b == c
h will be incremented until b == c
b is 17000 less than c initially in prod.
c doesn't change
b is incremented by 17 whenever h is incremented and the program doesn't terminate
_OR_ if f is not zero at the top of this sequence.
```



Considering seq:
```
...
sub e -1
set g e
sub g b
jnz g -8
sub d -1
set g d
sub g b
jnz g -13
jnz f 2
sub h -1
...

To get into the lower sequence g == 0 to skip jnz g -13
g == 0 => b == d
To get past jnz g -8 g==0
g == 0 => b == e
```

Considering the following sequence:

```
set f 1
set d 2
set e 2
set g d
mul g e    g = d * e - b
sub g b
jnz g 2
set f 0
sub e -1
set g e
sub g b
jnz g -8
sub d -1
set g d
sub g b
jnz g -13
```

It does the following:

In [9]:
def subroutine(b):
    f = 1
    d = 2
    while b != d:
        e = 2
        while b != e:
            if d * e - b == 0:
                f = 0
            e += 1
        d += 1
    return f   

In [10]:
[i for i in range(2,100) if subroutine(i) == 1]

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

In [11]:
# WTF - They're primes!!!!
#Program does the following:
def routine():
    b = 109900
    c = 126900
    h = 1
    while b != c:
        if not sympy.isprime(b):
            h += 1           
        b += 17
    return h   

In [12]:
routine()

913